In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy.optimize as opt
from lmfit import models

/home/frederik/anaconda3/lib/python3.4/site-packages/IPython/html.py:14: ShimWarning: The `IPython.html` package has been deprecated. You should import from `notebook` instead. `IPython.html.widgets` has moved to `ipywidgets`.
  "`IPython.html.widgets` has moved to `ipywidgets`.", ShimWarning)


In [2]:
pc_henrik = '/home/arrow13/Git/FP/Roentgen/'
pc_frederik = '/home/frederik/Dokumente/FP/Roentgen'

In [3]:
var_pc = pc_frederik

In [4]:
cd $var_pc/Messdaten

/home/frederik/Dokumente/FP/Roentgen/Messdaten


In [33]:
data = np.loadtxt("Si111_40_30.xy").T

# Einlesen, vorbereiten und auswerten der Daten (mit Versuchsplan)

## Messen des Röntgenspektrums der Kupferanode
Es werden drei verschiedene Spannungen und Ströme verwendet
+ U$_1$ = 30keV, A$_1$ = 10mA
+ U$_2$ = 40keV, A$_2$ = 10mA
+ U$_3$ = 40keV, A$_3$ = 30mA

Die letzte Einstellung wird noch mal mit einem Ni-Filter wiederholt

+ U$_{Ni}$ = 30keV, A$_{Ni}$ = 30mA

Auswertung:
+ Zählrate als Funktion des Winkels auftragen
+ Die lage der K$_{\alpha_{1,2}}$- und der K$_\beta$-Linien bestimmen, sowie die Verhältnisse (unter einander und unter den Ordnungen)
+ Mit Ni-Filter die Abschwächung der K$_\beta$-Linie, das "Signal-zu-Rausch" Verhältnis für die K$_{\alpha_{1,2}}$-Linie, die Energie und die Energiebreite bestimmen
+ Weitere Details des Spektrums diskutieren

## Netzebenabstand Si(331)
+ Aus dem Diffraktogramm soll der Netzebenabstand von Si(331) bestimmt werden
+ Der bestimmte Netzebenabstand soll mit Literaturwerten abgeglichen werden

## Netzebenabstand Ge(111)
+ Aus dem Diffraktogramm soll der Netzebenabstand von Si(331) bestimmt werden
+ Der bestimmte Netzebenabstand soll mit Literaturwerten abgeglichen werden

## Pulverdiraktometrie
+ Aus dem Diffraktogramm soll mittels einer Datenbank qualitativ die Zusammensetzung bestimmt werden
+ Unabhängig von der Datenbank sollen die Netzebenabstände des Pulvers bestimmt werden
+ Graphisch zeigen, dass die gefundene Kirstallstrucktur mit dem Diffraktogramm verträglich ist
+ Die mittlere Kristallgröße ermitteln

### Voigt für den einzel Peak 

In [70]:
intervall_voigt_1 = np.arange(np.where(data[0] > 25.62)[0][0], np.where(data[0] > 25.67)[0][0])
err_1 = np.sqrt(data[1][intervall_voigt_1])
err_1 = np.where(err_1 == 0, 1, err_1)

In [71]:
voigt_1 = models.VoigtModel(prefix='voigt_1')
pars_1 = voigt_1.guess(data[1][intervall_voigt_1],x=data[0][intervall_voigt_1])
pars_1['voigt_1gamma'].set(0.001,vary=True,expr='')

In [72]:
out_1 = voigt_1.fit(data[1][intervall_voigt_1],pars_1,x=data[0][intervall_voigt_1], weights=1/(err_1))

In [73]:
out_1.plot()
plt.errorbar(data[0][intervall_voigt_1],data[1][intervall_voigt_1],err_1,fmt='. ')
plt.show()

In [74]:
print(out_1.fit_report())

[[Model]]
    Model(voigt, prefix='voigt_1')
[[Fit Statistics]]
    # function evals   = 188
    # data points      = 12
    # variables        = 4
    chi-square         = 9.005
    reduced chi-square = 1.126
[[Variables]]
    voigt_1center:      25.6452178 +/- 6.76e-05 (0.00%) (init= 25.64408)
    voigt_1sigma:       0.03124682 +/- 0.005093 (16.30%) (init= 0.009504534)
    voigt_1amplitude:   498.119354 +/- 158.3703 (31.79%) (init= 487.6265)
    voigt_1fwhm:        0.11252948 +/- 0.018343 (16.30%)  == '3.6013100*voigt_1sigma'
    voigt_1gamma:      -0.03418473 +/- 0.016661 (48.74%) (init= 0.001)
[[Correlations]] (unreported correlations are <  0.100)
    C(voigt_1amplitude, voigt_1gamma)  =  1.000 
    C(voigt_1sigma, voigt_1gamma)  = -1.000 
    C(voigt_1sigma, voigt_1amplitude)  = -0.999 
    C(voigt_1center, voigt_1amplitude)  =  0.231 
    C(voigt_1center, voigt_1gamma)  =  0.228 
    C(voigt_1center, voigt_1sigma)  = -0.224 


### Mulit-Voigt

In [108]:
intervall_voigt_2 = np.arange(np.where(data[0] > 28.397)[0][0], np.where(data[0] > 28.54)[0][0])
guess_intervall_voigt_2 = np.arange(np.where( 28.39 < data[0])[0][0],np.where( data[0] > 28.48)[0][0])
guess_intervall_voigt_3 = np.arange(np.where( 28.47 < data[0])[0][0],np.where( data[0] > 28.54)[0][0])
err_2 = np.sqrt(data[1][intervall_voigt_2])
err_2 = np.where(err_2 == 0, 1, err_2)

In [109]:
voigt_2 = models.VoigtModel(prefix='voigt_2')
voigt_3 = models.VoigtModel(prefix='voigt_3')
dual_voigt = voigt_2 + voigt_3
pars_2 = voigt_2.guess(data[1][guess_intervall_voigt_2],x=data[0][guess_intervall_voigt_2])
pars_2.update(voigt_3.guess(data[1][guess_intervall_voigt_3],x=data[0][guess_intervall_voigt_3]))
pars_2['voigt_2gamma'].set(0.0001,vary=True,expr='')
pars_2['voigt_3gamma'].set(0.0001,vary=True,expr='')
pars_2['voigt_2amplitude'].set(1000)
pars_2['voigt_3amplitude'].set(1000)

In [110]:
out_2 = dual_voigt.fit(data[1][intervall_voigt_2],pars_2,x=data[0][intervall_voigt_2],weights=1/(err_2))

In [111]:
out_2.plot_fit()
plt.errorbar(data[0][intervall_voigt_2],data[1][intervall_voigt_2],err_2,fmt='. ')
plt.show()

In [113]:
print(out_2.fit_report())

[[Model]]
    (Model(voigt, prefix='voigt_2') + Model(voigt, prefix='voigt_3'))
[[Fit Statistics]]
    # function evals   = 192
    # data points      = 34
    # variables        = 8
    chi-square         = 1858.755
    reduced chi-square = 71.491
[[Variables]]
    voigt_2sigma:       0.04008803 +/- 0.014852 (37.05%) (init= 0.0149357)
    voigt_2amplitude:   1046.34036 +/- 872.8670 (83.42%) (init= 1000)
    voigt_2center:      28.4331814 +/- 0.001142 (0.00%) (init= 28.43486)
    voigt_2fwhm:        0.14436943 +/- 0.053490 (37.05%)  == '3.6013100*voigt_2sigma'
    voigt_2gamma:      -0.04797853 +/- 0.051231 (106.78%) (init= 0.0001)
    voigt_3amplitude:   1814.27400 +/- 613.5717 (33.82%) (init= 1000)
    voigt_3center:      28.5070649 +/- 0.001116 (0.00%) (init= 28.50588)
    voigt_3sigma:       0.02232692 +/- 0.005120 (22.94%) (init= 0.01222012)
    voigt_3fwhm:        0.08040616 +/- 0.018441 (22.94%)  == '3.6013100*voigt_3sigma'
    voigt_3gamma:      -0.00444114 +/- 0.012671 (285.33

In [114]:
plt.subplot(121)
plt.errorbar(data[0][intervall_voigt_1],data[1][intervall_voigt_1],err_1,fmt='. ',label="Messdaten")
plt.plot(data[0][intervall_voigt_1],out_1.best_fit,label="Voigt-Fit")
plt.ticklabel_format(useOffset=False)
plt.xticks(fontsize=30)
plt.yticks(fontsize=30)
plt.legend(loc='best',fontsize=42)
plt.xlabel('Winkel in 2$\Theta$ /$^\circ$',fontsize=42)
plt.ylabel('Counts',fontsize=42)
plt.title('Fit für den einzelnen Peak',fontsize=42)


plt.subplot(122)
plt.errorbar(data[0][intervall_voigt_2],data[1][intervall_voigt_2],err_2,fmt='. ',label="Messdaten")
plt.plot(data[0][intervall_voigt_2],out_2.best_fit,label="Multi-Voigt-Fit")
plt.ticklabel_format(useOffset=False)
plt.xlabel('Winkel in 2$\Theta$ /$^\circ$',fontsize=42)
plt.ylabel('Counts',fontsize=42)
plt.xticks(fontsize=30)
plt.yticks(fontsize=30)
plt.legend(loc='best',fontsize=42)
plt.title('Fit für die überlagerten Peaks',fontsize=42)

plt.show()

# Plots der Spektren für die ersten 4 Aufnahme

In [17]:
data_spektrum = np.loadtxt("Si111_40_30_Ni.xy").T